## Preprocessing Data
Preprocessing is inspired by the following paper
https://www.mdpi.com/2073-4433/13/10/1719

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv("beijing_air_quality_data.csv")
df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [4]:
df = df.drop(['station', 'year','month','day','hour'], axis=1)
df.head(5)

,No,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,1,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4
1,2,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7
2,3,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6
3,4,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1
4,5,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0


In [5]:
wd_unique_values = df['wd'].unique()
wd_unique_values

array(['NNW', 'N', 'NW', 'NNE', 'ENE', 'E', 'NE', 'W', 'SSW', 'WSW', 'SE',
       'WNW', 'SSE', 'ESE', 'S', 'SW', nan], dtype=object)

In [6]:
df['wd'].isnull().sum()

81

In [7]:
wd_reshaped = np.array(df['wd']).reshape(-1, 1)
wd_encoder = OneHotEncoder()
wd_values = wd_encoder.fit_transform(wd_reshaped)
wd = pd.DataFrame(wd_values.toarray(),columns=wd_unique_values)
wd.head(5)

,NNW,N,NW,NNE,ENE,E,NE,W,SSW,WSW,SE,WNW,SSE,ESE,S,SW,NaN
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_numeric = df.drop(['wd'], axis=1)
df_numeric.head(5)

,No,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,1,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,4.4
1,2,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
2,3,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
3,4,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
4,5,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0


In [9]:
df_numeric.isnull().sum()

No          0
PM2.5     925
PM10      718
SO2       935
NO2      1023
CO       1776
O3       1719
TEMP       20
PRES       20
DEWP       20
RAIN       20
WSPM       14
dtype: int64

In [14]:
for col in df_numeric.columns:
  df_numeric[col].interpolate(method="linear", limit_direction="both",inplace=True)
imputed_vals = df_numeric.copy(deep=True)

In [15]:
df_numeric_imputed = pd.DataFrame(imputed_vals,columns=df_numeric.columns)
df_numeric_imputed.head()

,No,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,1,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,4.4
1,2,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
2,3,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
3,4,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
4,5,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0


In [16]:
df_numeric_imputed.isnull().sum()

No       0
PM2.5    0
PM10     0
SO2      0
NO2      0
CO       0
O3       0
TEMP     0
PRES     0
DEWP     0
RAIN     0
WSPM     0
dtype: int64

In [30]:
df_numeric_scale_vals = df_numeric_imputed.drop(['No'], axis=1)
df_numeric_scale_vals.head(5)

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,4.4
1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0


In [31]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_numeric_scale_vals)

In [32]:
df_processed_numeric = pd.DataFrame(df_scaled,columns=df_numeric_scale_vals.columns)
df_processed_numeric.head(5)

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,0.001117,0.002037,0.010902,0.017361,0.020202,0.181619,0.280977,0.661319,0.258621,0.0,0.392857
1,0.005587,0.006110,0.010902,0.017361,0.020202,0.181619,0.273997,0.664884,0.268025,0.0,0.419643
2,0.004469,0.005092,0.013837,0.027778,0.020202,0.172158,0.273997,0.670232,0.268025,0.0,0.500000
3,0.003352,0.004073,0.031447,0.031250,0.020202,0.169792,0.268761,0.688057,0.249216,0.0,0.276786
4,0.000000,0.001018,0.034382,0.034722,0.020202,0.169792,0.258290,0.700535,0.247649,0.0,0.178571


In [33]:
df_processed = pd.concat([df_processed_numeric,wd],axis=1)
df_processed

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,W,SSW,WSW,SE,WNW,SSE,ESE,S,SW,NaN
0,0.001117,0.002037,0.010902,0.017361,0.020202,0.181619,0.280977,0.661319,0.258621,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.005587,0.006110,0.010902,0.017361,0.020202,0.181619,0.273997,0.664884,0.268025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.004469,0.005092,0.013837,0.027778,0.020202,0.172158,0.273997,0.670232,0.268025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.003352,0.004073,0.031447,0.031250,0.020202,0.169792,0.268761,0.688057,0.249216,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.001018,0.034382,0.034722,0.020202,0.169792,0.258290,0.700535,0.247649,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.010056,0.027495,0.013837,0.114583,0.030303,0.224193,0.511344,0.491979,0.299373,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35060,0.011173,0.035642,0.019707,0.149306,0.040404,0.191080,0.495637,0.493761,0.316614,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
35061,0.014525,0.035642,0.028512,0.222222,0.060606,0.136679,0.481675,0.504456,0.344828,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35062,0.020112,0.042770,0.034382,0.295139,0.060606,0.082278,0.476440,0.508021,0.351097,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Generating Events
Events represent continuous segments of time series data, each indicating a specific occurrence. Each event maintains a consistent duration. The goal is to predict the following event, which is considered the target, based on the input of the current one.

In [34]:
# define window size and shift size
window_size = 25
shift_size = 5

In [35]:
# Create input events and target events
data = df_processed.values
input_events, target_events = [], []

for i in range(0, len(data) - window_size - shift_size + 1, shift_size):
    input_window = data[i:i + window_size]
    output_window = data[i + window_size:i + window_size + window_size]
    if len(input_window) == window_size and len(output_window) == window_size:
        input_events.append(np.array(input_window))
        target_events.append(np.array(output_window))

input_events = np.array(input_events)
target_events = np.array(target_events)

## Event Dimensionality Reduction Using SOM
Self-Organizing Maps (SOM) serve as topological maps for dimensionality reduction, where each SOM index corresponds to an event. The creation of this topological map is guided by the Frobenius Norm as the distance metric, effectively organizing events into a simplified, lower-dimensional space while preserving their topological characteristics.

In [36]:
import numpy as np
from typing import Tuple, Optional

class MAP ():

    def __init__(self, shape: Tuple, dim: Tuple, weights_path: Optional[str] = None):
        self.shape = shape
        self.dim = dim
        if weights_path:
            self.weights = self.load_weights(weights_path)
        else:
            self.weights = np.random.random(shape + dim)

    def save_weights(self, filepath: str) -> None:
        flattened_weights = self.weights.reshape(-1, np.prod(self.dim))
        np.savetxt(filepath, flattened_weights, delimiter=",")

    def load_weights(self, filepath: str) -> np.ndarray:
        flattened_weights = np.loadtxt(filepath, delimiter=",")
        return flattened_weights.reshape(self.shape + self.dim)

    def interpolate(self, index: Tuple) -> np.ndarray:

        # Clamp index to the bounds of the weight matrix
        x_index, y_index = index
        x_index = max(0, min(x_index, self.shape[0] - 1))
        y_index = max(0, min(y_index, self.shape[1] - 1))

        # Calculate the indices of the corners
        x_floor, y_floor = np.floor([x_index, y_index]).astype(int)
        x_ceil, y_ceil = np.ceil([x_index, y_index]).astype(int)

        # If the index is at an exact location, return the weight at that index
        if (x_floor == x_ceil) and (y_floor == y_ceil):
            return self.weights[x_floor, y_floor]

        # Calculate the interpolation weights
        x_weight = x_index - x_floor
        y_weight = y_index - y_floor

        # Interpolate between the four surrounding points
        top_left = self.weights[x_floor, y_floor]
        top_right = self.weights[min(x_ceil, self.shape[0] - 1), y_floor]
        bottom_left = self.weights[x_floor, min(y_ceil, self.shape[1] - 1)]
        bottom_right = self.weights[min(x_ceil, self.shape[0] - 1), min(y_ceil, self.shape[1] - 1)]

        # Perform bilinear interpolation
        top_interp = (1 - x_weight) * top_left + x_weight * top_right
        bottom_interp = (1 - x_weight) * bottom_left + x_weight * bottom_right
        interpolated_weight = (1 - y_weight) * top_interp + y_weight * bottom_interp

        return interpolated_weight



class SOM(MAP):
    def __init__(self, shape: Tuple[int, int], dim: Tuple, sigma_i: float = 10.0, sigma_f: float = 0.01, lrate_i: float = 0.5, lrate_f: float = 0.005, weights_path: Optional[str] = None):
        super().__init__(shape, dim, weights_path)
        self.sigma_i, self.sigma_f = sigma_i, sigma_f
        self.lrate_i, self.lrate_f = lrate_i, lrate_f
        self.global_t = 0
        self.total_iterations = 0
    
    def get_bmu_sequences(self, data) -> np.ndarray:
        n_samples = data.shape[0]
        bmu_sequences = []
        indices = np.arange(n_samples)

        for idx in indices:
            input = data[idx]
            dist = np.linalg.norm(self.weights-input, axis=(2, 3))
            bmu_location = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
            bmu_sequences.append(bmu_location)
        
        return np.array(bmu_sequences)

    def learn(self, data: np.ndarray, epochs: int = 100, shuffle: bool = True, random_state=None) -> None:
        self.total_iterations = epochs * data.shape[0]
        n_samples = data.shape[0]

        for epoch in range(epochs):
            print(f'Epoch {epoch+1}/{epochs}', end='\r')
            if shuffle:
                rng = np.random.default_rng(random_state)
                indices = rng.permutation(n_samples) # Shuffle indices
            else:
                indices = np.arange(n_samples)

            for idx in indices:
                input = data[idx]
                self.step(input) # Do one step of training

    def step(self, x: np.ndarray) -> None:

        dist = np.linalg.norm(self.weights-x, axis=(2, 3))
        bmu_location = np.unravel_index(np.argmin(dist, axis=None), dist.shape) # best matching unit location
        dist_to_bmu = np.linalg.norm(np.stack(np.indices(self.shape), axis=-1) - bmu_location, axis=-1)  # distance from the best matching unit to all other nodes in the map

        # update learning rate and sigma values
        t = self.global_t/self.total_iterations
        lrate = self.lrate_i*(self.lrate_f/self.lrate_i)**t
        sigma = self.sigma_i*(self.sigma_f/self.sigma_i)**t

        # update weights
        eta = lrate * np.exp(-1*dist_to_bmu**2 /(2 * sigma**2))
        self.weights += eta[:, :, np.newaxis, np.newaxis] * (x - self.weights)

        self.global_t += 1

## Train SOM and save weights

In [37]:
input_dim = (window_size, 28)
som_dim = (2**6,2**6)
som = SOM(shape = som_dim, dim=input_dim,lrate_i=1,sigma_i=max(som_dim)/2)
som.learn(input_events,100)
som.save_weights('som_weights.csv')

## Load weights and initialize SOM

In [38]:
input_dim = (window_size, 28)
som_dim = (2**6,2**6)
som = SOM(shape = som_dim, dim=input_dim,lrate_i=1,sigma_i=max(som_dim)/2,weights_path='som_weights.csv')

## Creating Dimensionally Reduced Dataset
Each input event is identified by the closest corresponding unit of the Self-Organizing Map (SOM), known as the best matching unit (BMU). Instead of using raw events, sequences of these BMUs are employed to train the Artificial Neural Network (ANN), effectively reducing the dataset's dimensionality while preparing it for efficient learning.

In [39]:
input = som.get_bmu_sequences(input_events)
target = som.get_bmu_sequences(target_events)

In [40]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Define the model with additional hidden layers
model = Sequential([
    Dense(units=64, activation='sigmoid', input_shape=(2,)), # Input layer
    Dense(units=128, activation='sigmoid'),
    Dense(units=2)  # Output layer with 2 neurons
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [41]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_targets, test_targets = train_test_split(input, target, test_size=0.2, random_state=42)

In [42]:
model.fit(train_inputs,train_targets , epochs=200)
test_loss = model.evaluate(test_inputs, test_targets)
print('Test Loss:', test_loss)

Epoch 1/200
176/176 [==============================] - 0s 433us/step - loss: 827.4197
Epoch 2/200
176/176 [==============================] - 0s 413us/step - loss: 411.1072
Epoch 3/200
176/176 [==============================] - 0s 534us/step - loss: 351.8634
Epoch 4/200
176/176 [==============================] - 0s 449us/step - loss: 325.6408
Epoch 5/200
176/176 [==============================] - 0s 413us/step - loss: 312.4080
Epoch 6/200
176/176 [==============================] - 0s 403us/step - loss: 295.4458
Epoch 7/200
176/176 [==============================] - 0s 407us/step - loss: 272.1491
Epoch 8/200
176/176 [==============================] - 0s 405us/step - loss: 258.5177
Epoch 9/200
176/176 [==============================] - 0s 402us/step - loss: 252.7310
Epoch 10/200
176/176 [==============================] - 0s 413us/step - loss: 249.7074
Epoch 11/200
176/176 [==============================] - 0s 401us/step - loss: 248.0738
Epoch 12/200
176/176 [==============================

In [43]:
predicted = model.predict(test_inputs)
predicted_weight = np.array([som.interpolate(pred) for pred in predicted])
target_weight = np.array([som.interpolate(target) for target in test_targets])

def mean_absolute_error(matrix1, matrix2):
    # Convert the matrices to NumPy arrays if they aren't already
    matrix1 = np.array(matrix1)
    matrix2 = np.array(matrix2)

    # Calculate the absolute differences
    absolute_differences = np.abs(matrix1 - matrix2)

    # Calculate the mean of the absolute differences
    mae = np.mean(absolute_differences)
    return mae

MAE = np.mean([mean_absolute_error(predicted_weight[i], target_weight[i]) for i in range(len(predicted_weight))])
print(f'Mean Absolute Error: {MAE}')


44/44 [==============================] - 0s 276us/step
Mean Absolute Error: 0.0923031153640831
